# Correlation

## Load data after handling null

In [21]:
from data_pipeline.open_data_raw_material_price.core import OpenDataRawMaterialPrice
from data_pipeline.open_data_marine_weather.core import OpenDataMarineWeather
from data_pipeline.open_data_terrestrial_weather.core import OpenDataTerrestrialWeather


t = OpenDataTerrestrialWeather(
    date="201908"
)
t_df = t.clean(t.filter(t.input_df))

m = OpenDataMarineWeather(
    date="201908"
)
m_df = m.clean(m.filter(m.input_df))

p = OpenDataRawMaterialPrice(
    date="201908"
)
p_df = p.clean(p.filter(p.input_df))

p_df.head(5)

2020-04-01 20:22:24:manager:<INFO> 1 files is loaded from public_data/open_data_terrestrial_weather/origin/csv/2014-2020.csv in s3 'production-bobsim'
2020-04-01 20:22:24:core:<INFO> Series([], dtype: int64)
2020-04-01 20:22:25:manager:<INFO> 1 files is loaded from public_data/open_data_marine_weather/origin/csv/2014-2020.csv in s3 'production-bobsim'
2020-04-01 20:22:25:core:<INFO> Series([], dtype: int64)
2020-04-01 20:22:33:manager:<INFO> 1 files is loaded from public_data/open_data_raw_material_price/origin/csv/201908.csv in s3 'production-bobsim'
2020-04-01 20:22:33:core:<INFO> no missing value at raw material price


,조사일자,조사지역명,품목명,당일조사가격
0,2019-08-01,강원,감자감자수미수미,238.0
1,2019-08-01,강원,건제품건포도건포도(수입)수입,1450.0
2,2019-08-01,강원,고추붉은고추홍고추일반,990.0
3,2019-08-01,강원,고추풋고추꽈리고추꽈리고추,1050.0
4,2019-08-01,강원,고추풋고추청양청양고추,670.0


## Merge 3 DataFrames into one df

In [22]:
import pandas as pd

w_df = pd.merge(
            t_df, m_df,
            how='inner', on="일시"
        )

w_df.head(5)

,일시,평균기온(°C),최저기온(°C),최고기온(°C),최대 풍속(m/s),평균 풍속(m/s)_x,최소 상대습도(pct),평균 상대습도(pct)_x,강수 계속시간(hr),일강수량(mm),...,평균기압(hPa),평균 상대습도(pct)_y,평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec)
0,2019-08-01,28.048422,24.331579,32.701054,3.921053,1.669474,58.642105,79.241051,4.1000,11.830770,...,1011.594116,91.0625,25.694117,24.729412,0.882353,0.529412,0.735294,1.252941,4.458824,5.547059
1,2019-08-02,28.432978,24.024467,33.870525,3.297895,1.243617,52.852631,77.096809,1.5650,0.720000,...,1011.549988,88.8125,26.382353,25.523529,0.564706,0.335294,0.500000,0.841176,4.535294,7.100000
2,2019-08-03,27.950527,23.901052,33.782104,3.910526,1.288421,54.673683,79.117897,1.0100,3.208333,...,1011.170593,90.6875,26.105883,25.711765,0.629412,0.375000,0.529412,0.929412,5.158823,8.123529
3,2019-08-04,28.063158,23.549475,33.776840,3.334737,1.203158,53.315788,77.867371,1.1725,3.863636,...,1010.593323,88.5000,26.468750,26.006250,0.700000,0.413333,0.558824,1.041176,5.231250,9.011765
4,2019-08-05,28.396843,23.751579,34.145264,3.453684,1.265263,52.536842,76.692635,0.8800,3.430000,...,1010.106262,88.6250,26.406250,26.218750,0.670588,0.400000,0.594118,1.076471,5.058824,7.905882


In [24]:
origin_df = pd.merge(
                p_df, w_df, how="inner", left_on="조사일자", right_on="일시"
            ).drop("일시", axis=1).astype(dtype={"조사일자": "datetime64"})

In [25]:
origin_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40512 entries, 0 to 40511
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   조사일자            40512 non-null  datetime64[ns]
 1   조사지역명           40512 non-null  object        
 2   품목명             40512 non-null  object        
 3   당일조사가격          40512 non-null  float64       
 4   평균기온(°C)        40512 non-null  float32       
 5   최저기온(°C)        40512 non-null  float32       
 6   최고기온(°C)        40512 non-null  float32       
 7   최대 풍속(m/s)      40512 non-null  float32       
 8   평균 풍속(m/s)_x    40512 non-null  float32       
 9   최소 상대습도(pct)    40512 non-null  float32       
 10  평균 상대습도(pct)_x  40512 non-null  float32       
 11  강수 계속시간(hr)     40512 non-null  float32       
 12  일강수량(mm)        40512 non-null  float32       
 13  평균 풍속(m/s)_y    40512 non-null  float32       
 14  평균기압(hPa)       40512 non-null  float32       
 15  평균

In [26]:
origin_df.describe()

,당일조사가격,평균기온(°C),최저기온(°C),최고기온(°C),최대 풍속(m/s),평균 풍속(m/s)_x,최소 상대습도(pct),평균 상대습도(pct)_x,강수 계속시간(hr),일강수량(mm),...,평균기압(hPa),평균 상대습도(pct)_y,평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec)
count,40512.000000,40512.000000,40512.000000,40512.000000,40512.000000,40512.000000,40512.000000,40512.000000,40512.000000,40512.000000,...,40512.000000,40512.000000,40512.000000,40512.000000,40512.000000,40512.000000,40512.000000,40512.000000,40512.000000,40512.000000
mean,1986.640772,26.024052,22.348356,30.675423,4.011300,1.586215,58.717220,80.291122,3.482316,7.611833,...,1006.944214,85.207024,26.038477,25.957531,1.354981,0.812570,1.122269,1.930990,5.802956,8.219999
std,2562.316177,2.048313,2.001055,2.492165,0.575702,0.358118,7.272638,4.331572,2.538251,8.130293,...,3.995971,4.824147,0.866968,0.455116,0.474614,0.290527,0.375879,0.613369,1.031278,1.497837
min,31.000000,22.217022,18.348936,25.268085,3.297895,1.146809,47.148937,73.802132,0.601000,0.006061,...,996.221436,71.647057,24.023529,24.729412,0.564706,0.335294,0.500000,0.841176,4.458824,5.547059
25%,572.000000,24.441935,20.613979,29.620213,3.453684,1.265263,52.852631,77.164520,1.491111,1.559259,...,1006.329407,83.785713,25.694117,25.623529,1.068750,0.637500,0.847059,1.470588,5.058824,7.100000
50%,1192.000000,26.171276,23.086170,30.641489,3.893617,1.457447,57.659573,78.819359,2.589444,3.430000,...,1007.588257,86.666664,26.333334,26.143749,1.252941,0.758824,1.117647,1.900000,5.587500,7.905882
75%,2233.000000,28.048422,23.974468,32.701054,4.463158,1.822581,61.659573,83.605316,4.585500,11.830770,...,1009.358826,88.625000,26.570587,26.278572,1.580000,0.953333,1.350000,2.287500,6.393750,9.417647
max,24900.000000,28.432978,25.078724,34.145264,5.101064,2.368085,75.500000,90.443619,9.144375,29.143749,...,1012.285706,91.062500,27.100000,26.500000,2.537500,1.562500,2.082353,3.388235,8.042857,11.652942


In [69]:
numeric_df = origin_df.select_dtypes(exclude=['object', 'datetime64[ns]'])
numeric_df.head(5)

,당일조사가격,평균기온(°C),최저기온(°C),최고기온(°C),최대 풍속(m/s),평균 풍속(m/s)_x,최소 상대습도(pct),평균 상대습도(pct)_x,강수 계속시간(hr),일강수량(mm),...,평균기압(hPa),평균 상대습도(pct)_y,평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec)
0,238.0,28.048422,24.331579,32.701054,3.921053,1.669474,58.642105,79.241051,4.1,11.83077,...,1011.594116,91.0625,25.694117,24.729412,0.882353,0.529412,0.735294,1.252941,4.458824,5.547059
1,1450.0,28.048422,24.331579,32.701054,3.921053,1.669474,58.642105,79.241051,4.1,11.83077,...,1011.594116,91.0625,25.694117,24.729412,0.882353,0.529412,0.735294,1.252941,4.458824,5.547059
2,990.0,28.048422,24.331579,32.701054,3.921053,1.669474,58.642105,79.241051,4.1,11.83077,...,1011.594116,91.0625,25.694117,24.729412,0.882353,0.529412,0.735294,1.252941,4.458824,5.547059
3,1050.0,28.048422,24.331579,32.701054,3.921053,1.669474,58.642105,79.241051,4.1,11.83077,...,1011.594116,91.0625,25.694117,24.729412,0.882353,0.529412,0.735294,1.252941,4.458824,5.547059
4,670.0,28.048422,24.331579,32.701054,3.921053,1.669474,58.642105,79.241051,4.1,11.83077,...,1011.594116,91.0625,25.694117,24.729412,0.882353,0.529412,0.735294,1.252941,4.458824,5.547059


## skew

In [71]:
from scipy.stats import skew

def get_skews(df):
    return df.apply(lambda x : skew(x))

skews = get_skews(numeric_df)
skews.reset_index()

,index,0
0,당일조사가격,3.452630
1,평균기온(°C),-0.449521
2,최저기온(°C),-0.560308
3,최고기온(°C),-0.641832
4,최대 풍속(m/s),0.341649
5,평균 풍속(m/s)_x,0.750571
6,최소 상대습도(pct),0.565824
7,평균 상대습도(pct)_x,0.673417
8,강수 계속시간(hr),0.864804
9,일강수량(mm),1.109568


## correlation

In [72]:
# default: method=pearson, min_periods=1
origin_corr = origin_df.corr()
origin_corr

,당일조사가격,평균기온(°C),최저기온(°C),최고기온(°C),최대 풍속(m/s),평균 풍속(m/s)_x,최소 상대습도(pct),평균 상대습도(pct)_x,강수 계속시간(hr),일강수량(mm),...,평균기압(hPa),평균 상대습도(pct)_y,평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec)
당일조사가격,1.000000,0.004224,0.003827,0.004260,0.000887,0.001575,-0.002179,-0.002047,-0.000971,0.000800,...,-0.001827,0.002173,0.005728,0.004594,0.001641,0.001590,0.002362,0.002259,0.002504,0.001566
평균기온(°C),0.004224,1.000000,0.917080,0.953130,0.020307,0.219314,-0.214687,-0.336246,-0.216775,-0.020111,...,-0.225121,0.748279,0.859464,0.289213,-0.116140,-0.116651,-0.043805,-0.064697,-0.047098,-0.043742
최저기온(°C),0.003827,0.917080,1.000000,0.765276,0.181040,0.313786,0.145893,0.008816,0.077529,0.246346,...,-0.409573,0.877907,0.796704,0.253805,0.080749,0.076647,0.181438,0.169231,-0.088418,-0.026087
최고기온(°C),0.004260,0.953130,0.765276,1.000000,-0.082631,0.110975,-0.474565,-0.537910,-0.423264,-0.186918,...,-0.055251,0.567704,0.818471,0.273425,-0.277868,-0.276263,-0.224014,-0.248916,-0.037213,-0.066003
최대 풍속(m/s),0.000887,0.020307,0.181040,-0.082631,1.000000,0.914111,0.194702,0.065473,0.270839,0.361052,...,-0.624432,-0.023524,0.136516,-0.043681,0.569320,0.569585,0.573384,0.581659,-0.136585,-0.153671
평균 풍속(m/s)_x,0.001575,0.219314,0.313786,0.110975,0.914111,1.000000,0.078678,-0.137145,0.187728,0.214715,...,-0.696714,0.064356,0.342552,0.092757,0.626125,0.629529,0.609971,0.605100,-0.082954,-0.178203
최소 상대습도(pct),-0.002179,-0.214687,0.145893,-0.474565,0.194702,0.078678,1.000000,0.927905,0.747128,0.554327,...,-0.391764,0.417136,-0.246801,-0.073009,0.492239,0.479784,0.520868,0.548039,0.107579,0.231472
평균 상대습도(pct)_x,-0.002047,-0.336246,0.008816,-0.537910,0.065473,-0.137145,0.927905,1.000000,0.755948,0.620167,...,-0.206867,0.329055,-0.365689,-0.152193,0.376840,0.360558,0.416328,0.443183,0.172186,0.302437
강수 계속시간(hr),-0.000971,-0.216775,0.077529,-0.423264,0.270839,0.187728,0.747128,0.755948,1.000000,0.871247,...,-0.178331,0.321444,-0.167841,-0.136974,0.500234,0.494389,0.526793,0.539405,0.125613,0.222261
일강수량(mm),0.000800,-0.020111,0.246346,-0.186918,0.361052,0.214715,0.554327,0.620167,0.871247,1.000000,...,-0.105799,0.401890,0.011139,-0.111531,0.300928,0.290059,0.367254,0.380990,-0.079546,0.074180


### correlation btw X and y

In [74]:
corr_target = abs(origin_corr["당일조사가격"])
corr_target.reset_index()

,index,당일조사가격
0,당일조사가격,1.000000
1,평균기온(°C),0.004224
2,최저기온(°C),0.003827
3,최고기온(°C),0.004260
4,최대 풍속(m/s),0.000887
5,평균 풍속(m/s)_x,0.001575
6,최소 상대습도(pct),0.002179
7,평균 상대습도(pct)_x,0.002047
8,강수 계속시간(hr),0.000971
9,일강수량(mm),0.000800


### sum of correlation coef

In [75]:
corr_target.drop("당일조사가격").sum()

0.04754771873052942

## Grid search for normalization

In [76]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler
import numpy as np

log = lambda df: np.log1p(df)
sqrt = lambda df: np.sqrt(df)
min_max = lambda df: MinMaxScaler(df)
quantile = lambda df: QuantileTransformer(df)
boxcos = lambda df: PowerTransformer(df, method='box-cos')
yeo_johnson = lambda df: PowerTransformer(df, method='yeo-johnson')

transformers = [log, sqrt, min_max, quantile, boxcos, yeo_johnson]

In [82]:
min_max(numeric_df)

MinMaxScaler(copy=True,
             feature_range=            당일조사가격   평균기온(°C)   최저기온(°C)   최고기온(°C)  최대 풍속(m/s)  평균 풍속(m/s)_x  \
0       238.000000  28.048422  24.331579  32.701054    3.921053      1.669474   
1      1450.000000  28.048422  24.331579  32.701054    3.921053      1.669474   
2       990.000000  28.048422  24.331579  32.701054    3.921053      1.669474   
3      1050.000000  28.048422  24.331579  32.701054    3.921053      1.669474   
4       670.000000  28.048422  24.331579  32.701054    3.9210...
1         0.735294     1.252941     4.458824     5.547059  
2         0.735294     1.252941     4.458824     5.547059  
3         0.735294     1.252941     4.458824     5.547059  
4         0.735294     1.252941     4.458824     5.547059  
...            ...          ...          ...          ...  
40507     0.976471     1.647059     4.723529     7.070588  
40508     0.976471     1.647059     4.723529     7.070588  
40509     0.976471     1.647059     4.723529     7.070588  

In [84]:
import functools

def analyze(func, df):
    processed_df = func(df)
    print(processed_df)
    if isinstance()
    return get_skews(processed_df)
    
result = list(map(functools.partial(analyze, df=numeric_df), transformers))
result

         당일조사가격  평균기온(°C)  최저기온(°C)  최고기온(°C)  최대 풍속(m/s)  평균 풍속(m/s)_x  \
0      5.476464  3.368964  3.232052  3.517529    1.593522      0.981881   
1      7.280008  3.368964  3.232052  3.517529    1.593522      0.981881   
2      6.898715  3.368964  3.232052  3.517529    1.593522      0.981881   
3      6.957497  3.368964  3.232052  3.517529    1.593522      0.981881   
4      6.508769  3.368964  3.232052  3.517529    1.593522      0.981881   
...         ...       ...       ...       ...         ...           ...   
40507  7.356918  3.163273  2.962637  3.365129    1.747348      1.072021   
40508  7.062192  3.163273  2.962637  3.365129    1.747348      1.072021   
40509  6.572283  3.163273  2.962637  3.365129    1.747348      1.072021   
40510  6.965080  3.163273  2.962637  3.365129    1.747348      1.072021   
40511  7.609697  3.163273  2.962637  3.365129    1.747348      1.072021   

       최소 상대습도(pct)  평균 상대습도(pct)_x  강수 계속시간(hr)  일강수량(mm)  ...  평균기압(hPa)  \
0          4.088362  

ValueError: DataFrame constructor not properly called!